In [3]:
import pandas as pd 

In [4]:
import numpy as np

In [5]:
!ls ../out

output_aCM.csv		output_FCHL.csv		 output_SLATM.csv
output_amons_aCM.csv	output_FCHL_global.csv	 output_SLATM_global.csv
output_amons_FCHL.csv	output_global_CM.csv	 output_SOAP.csv
output_amons_SLATM.csv	output_global_FCHL.csv	 output_SOAP_global.csv
output_amons_SOAP.csv	output_global_SLATM.csv  output_SPAHM_global.csv
output_CM_global.csv	output_global_SPAHM.csv


In [6]:
slatm_out = pd.read_csv("../out/output_SOAP_global.csv", index_col=0)

In [7]:
slatm_out

,SolN,Fragments,ObjValNoPen,ObjValWithPen
0,1,"['frag_06_c00001', 'frag_14_c00001']",0.043947,0.043947
1,2,"['frag_08_c00001', 'frag_12_c00001']",0.046057,0.046057
2,3,"['frag_08_c00001', 'frag_11_c00001']",0.046654,0.046654
3,4,"['frag_10_c00001', 'frag_12_c00001']",0.047774,0.047774
4,5,"['frag_06_c00001', 'frag_13_c00001']",0.048499,0.048499
...,...,...,...,...
995,996,"['frag_02_c00001', 'frag_06_c00001', 'frag_07_...",0.505390,5.505390
996,997,"['frag_04_c00001', 'frag_05_c00001', 'frag_10_...",0.506760,5.506760
997,998,"['frag_02_c00001', 'frag_06_c00001', 'frag_07_...",0.507137,5.507137
998,999,"['frag_02_c00001', 'frag_05_c00001', 'frag_06_...",0.507892,5.507892


In [8]:
from ast import literal_eval 

In [9]:
slatm_out["Fragments"] = slatm_out["Fragments"].apply(literal_eval)

In [10]:
slatm_out["Sorted Fragments"] = slatm_out["Fragments"].apply(lambda x:sorted(x))

In [11]:
len(np.unique(slatm_out["Sorted Fragments"]))

1000

In [12]:
# excess is missing Hs - to fix 

In [13]:
import qml

In [14]:
qm9_mol = qml.Compound("../targets/qm9_0.xyz")
qm9_ncharges = qm9_mol.nuclear_charges

In [15]:
qm9_ncharges

array([8, 6, 6, 7, 6, 8, 8, 7, 6, 1, 1])

In [16]:
def count_ncharges(list_ncharges, elements=[1,6,7,8,16]):
    ncharge_count = np.zeros(len(elements))
    for ncharge in list_ncharges:
        for i, elem in enumerate(elements):
            if ncharge == elem:
                ncharge_count[i] += 1
    return ncharge_count

In [17]:
qm9_counts = count_ncharges(qm9_ncharges)

In [18]:
qm9_counts

array([2., 4., 2., 3., 0.])

In [19]:
def get_excess_frags(frag_list):
    mols = [qml.Compound("../amons-qm9/"+x+".xyz") for x in frag_list]
    ncharges = np.concatenate([mol.nuclear_charges for mol in mols])
    ncharge_count = count_ncharges(ncharges)
    
    diff_count = ncharge_count - qm9_counts
    excess = [[ncharge]*int(count) for ncharge, count in zip([1,6,7,8,16], diff_count)]
    return np.concatenate(excess)

In [20]:
def get_excess_count(frag_list):
    mols = [qml.Compound("../amons-qm9/"+x+".xyz") for x in frag_list]
    ncharges = np.concatenate([mol.nuclear_charges for mol in mols])
    ncharge_count = count_ncharges(ncharges)
    
    diff_count = ncharge_count - qm9_counts
    return diff_count

In [21]:
slatm_out["Excess"] = slatm_out["Fragments"].apply(get_excess_frags)

In [26]:
slatm_out.sort_values(by='ObjValWithPen').iloc[0:50]

,SolN,Fragments,ObjValNoPen,ObjValWithPen,Sorted Fragments,Excess,Excess Counts
0,1,"[frag_06_c00001, frag_14_c00001]",0.043947,0.043947,"[frag_06_c00001, frag_14_c00001]","[1.0, 1.0, 1.0, 1.0]","[4.0, 0.0, 0.0, 0.0, 0.0]"
1,2,"[frag_08_c00001, frag_12_c00001]",0.046057,0.046057,"[frag_08_c00001, frag_12_c00001]","[1.0, 1.0, 1.0, 1.0]","[4.0, 0.0, 0.0, 0.0, 0.0]"
2,3,"[frag_08_c00001, frag_11_c00001]",0.046654,0.046654,"[frag_08_c00001, frag_11_c00001]","[1.0, 1.0, 1.0, 1.0]","[4.0, 0.0, 0.0, 0.0, 0.0]"
3,4,"[frag_10_c00001, frag_12_c00001]",0.047774,0.047774,"[frag_10_c00001, frag_12_c00001]","[1.0, 1.0, 1.0, 1.0]","[4.0, 0.0, 0.0, 0.0, 0.0]"
4,5,"[frag_06_c00001, frag_13_c00001]",0.048499,0.048499,"[frag_06_c00001, frag_13_c00001]","[1.0, 1.0, 1.0, 1.0, 1.0, 7.0]","[5.0, 0.0, 1.0, -1.0, 0.0]"
5,6,"[frag_10_c00001, frag_11_c00001]",0.050336,0.050336,"[frag_10_c00001, frag_11_c00001]","[1.0, 1.0, 1.0, 1.0]","[4.0, 0.0, 0.0, 0.0, 0.0]"
6,7,"[frag_05_c00001, frag_13_c00001]",0.053927,0.053927,"[frag_05_c00001, frag_13_c00001]","[1.0, 1.0, 1.0, 1.0]","[4.0, 0.0, 0.0, 0.0, 0.0]"
7,8,"[frag_03_c00001, frag_15_c00001]",0.054130,0.054130,"[frag_03_c00001, frag_15_c00001]","[1.0, 1.0, 1.0, 1.0]","[4.0, 0.0, 0.0, 0.0, 0.0]"
8,9,"[frag_07_c00001, frag_12_c00001]",0.056099,0.056099,"[frag_07_c00001, frag_12_c00001]","[1.0, 1.0, 1.0, 1.0, 1.0, 7.0]","[5.0, 0.0, 1.0, -1.0, 0.0]"
9,10,"[frag_04_c00001, frag_16_c00001]",0.056688,0.056688,"[frag_04_c00001, frag_16_c00001]","[1.0, 1.0]","[2.0, 0.0, 0.0, 0.0, 0.0]"
